In [2]:
import numpy as np, pandas as pd, time, os, subprocess, scipy as sp, re, seaborn as sns
import app_config, importlib, app_lib
import diffmap as dm
from sklearn.metrics.pairwise import euclidean_distances

# plot_data_df = all_data.obs
# plot_data_df[app_config.params['display_coordinates']['x']] = all_data.obsm['X_umap'][:,0]
# plot_data_df[app_config.params['display_coordinates']['y']] = all_data.obsm['X_umap'][:,1]

In [3]:
plot_data_df = pd.read_csv(app_config.params['plot_data_df_path'][0], sep="\t", index_col=False)
plot_data_df

,EM_x,EM_y,EM_z,tSNE_x,tSNE_y,hEM_x,hEM_y,hEM_z,htSNE_x,htSNE_y,hUMAP_x,hUMAP_y,roarke_clusters,top_enriched_clusters,gene_names,max_enrichments
0,0.000386,0.000528,0.000687,-36.286868,46.840289,0.000124,0.000136,0.000166,-15.135076,55.099715,2.957289,9.025891,Unknown,Unknown,A1BG,4.190389
1,0.000372,0.000353,0.000509,-0.026093,39.027340,0.000119,0.000091,0.000123,38.991786,10.915805,5.869159,-2.235847,Unknown,Unknown,A1CF,2.505144
2,0.000284,0.000252,0.000120,-5.460029,2.170856,0.000091,0.000065,0.000029,-1.451249,1.268335,2.488397,-1.170052,Unknown,Unknown,A2M,0.000000
3,0.000309,-0.025158,0.000982,-7.412075,-30.817777,0.000099,-0.006482,0.000237,5.171936,-71.515775,-4.953838,14.731494,Unknown,Unknown,A2ML1,0.000000
4,0.000343,0.000272,0.000418,3.999725,5.142851,0.000110,0.000070,0.000101,3.114053,-4.136086,2.570755,-1.473688,Unknown,Unknown,A3GALT2,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17629,0.000335,0.000303,0.000413,-11.454598,-36.487676,0.000108,0.000078,0.000100,-2.493834,12.193343,-0.504605,0.005887,Unknown,Unknown,ZYG11A,0.000000
17630,0.000527,-0.000407,0.000147,41.116041,50.268671,0.000169,-0.000105,0.000035,43.179570,55.526012,7.782668,13.050512,Unknown,Unknown,ZYG11B,3.413896
17631,0.000089,0.000205,0.000488,0.862270,-23.636117,0.000029,0.000053,0.000118,16.793142,-33.694676,4.512137,-2.932982,Unknown,Unknown,ZYX,0.000000
17632,0.000296,0.000362,0.000327,-10.826518,-14.760883,0.000095,0.000093,0.000079,-15.256799,3.355386,3.436330,0.419711,Unknown,Unknown,ZZEF1,2.865524


In [13]:
#data_ess = pd.read_csv(app_config.params['raw_ess_data_path'], index_col=0, header=0, sep='\t')
data_ess = data_ess[data_ess.columns[:-4]]   # Only first 481 cols put through GLS, so isolate these

In [27]:
for x in data_ess.loc['EML2'].index:
    if 'M059' in x:
        print(x)

M059K_CENTRAL_NERVOUS_SYSTEM


In [31]:
data_ess.loc['HNRNPCL1']['CME1_SOFT_TISSUE']

-0.094369144856348

In [6]:
from goatools.associations import read_ncbi_gene2go
go2geneids_human = read_ncbi_gene2go(app_config.params['gene2go_path'], taxids=[9606], go2geneids=True)

DEPRECATED read_ncbi_gene2go: USE Gene2GoReader FROM goatools.anno.genetogo_reader
DEPRECATED read_ncbi_gene2go CALLED FROM: <ipython-input-6-d60b36872896> BY <module>
HMS:0:00:04.589093 323,125 annotations READ: /Users/akshay/github/coessentiality-browser/gene2go 
1 taxids stored: 9606
12285 IDs in loaded association branch, BP


['GO:0002576',
 'GO:0043312',
 'GO:0010951',
 'GO:0007597',
 'GO:0051056',
 'GO:0001869',
 'GO:0048863',
 'GO:0022617',
 'GO:0006805',
 'GO:0030277',
 'GO:0019216',
 'GO:0006953',
 'GO:0006954',
 'GO:0009056',
 'GO:0010898',
 'GO:0030154',
 'GO:0014909',
 'GO:0010595',
 'GO:0001525',
 'GO:0009648',
 'GO:0071320',
 'GO:0010043',
 'GO:0030187',
 'GO:0034097',
 'GO:0032868',
 'GO:0009416',
 'GO:0046219',
 'GO:0007623',
 'GO:0034695',
 'GO:0006474',
 'GO:0051412',
 'GO:0046688',
 'GO:0051592',
 'GO:0008033',
 'GO:0070143',
 'GO:0106074',
 'GO:0006418',
 'GO:0006400',
 'GO:0006419',
 'GO:0050885',
 'GO:0021680',
 'GO:0140018',
 'GO:0043524',
 'GO:0010039',
 'GO:0007568',
 'GO:0035640',
 'GO:0097151',
 'GO:0045471',
 'GO:0031652',
 'GO:0014053',
 'GO:0048148',
 'GO:0042135',
 'GO:0001666',
 'GO:0070474',
 'GO:0090331',
 'GO:0007626',
 'GO:0009450',
 'GO:0032024',
 'GO:0033602',
 'GO:1902722',
 'GO:0035094',
 'GO:0021549',
 'GO:0045776',
 'GO:0007620',
 'GO:0045964',
 'GO:1904450',
 'GO:00094

In [64]:
import goterm_caller, json
srchhelp = goterm_caller.GoSearch(app_config.params['go_obo_path'], go2items=go2geneids_human)

/Users/akshay/github/coessentiality-browser/data/go-basic.obo: fmt(1.2) rel(2018-09-19) 47,343 GO Terms; optional_attrs(comment def relationship synonym xref)


In [49]:
%timeit a = srchhelp.get_items(['GO:0000003'])

552 ns ± 12.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


{2796, 2797, 8510, 286826}

In [58]:
toret['GO:0000003']

['LIN9', 'GNRH1', 'GNRH2', 'MMP23B']

In [61]:
from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT
goterm_to_gene = {}
go2geneids_human = read_ncbi_gene2go(app_config.params['gene2go_path'], taxids=[9606], go2geneids=True)
srchhelp = goterm_caller.GoSearch(app_config.params['go_obo_path'], go2items=go2geneids_human)
for got in go2geneids_human.keys():
    goterm_to_gene[got] = []
    for gid in srchhelp.get_items([got]):
        geneID = GENEID2NT.get(gid, None)
        if geneID is not None:
            goterm_to_gene[got].append(geneID.Symbol)

w = json.dumps(goterm_to_gene)
with open("go2gene_dict.txt", "w") as text_file:
    print(json.dumps(goterm_to_gene), file=text_file)

DEPRECATED read_ncbi_gene2go: USE Gene2GoReader FROM goatools.anno.genetogo_reader
DEPRECATED read_ncbi_gene2go CALLED FROM: <ipython-input-61-584bce70ae5a> BY <module>
HMS:0:00:04.199199 323,125 annotations READ: /Users/akshay/github/coessentiality-browser/gene2go 
1 taxids stored: 9606
12285 IDs in loaded association branch, BP
/Users/akshay/github/coessentiality-browser/data/go-basic.obo: fmt(1.2) rel(2018-09-19) 47,343 GO Terms; optional_attrs(comment def relationship synonym xref)


In [83]:
with open("go2gene_dict.txt", "r") as f:
    w2 = f.read().strip()
go2gene_dict = json.loads(w2)

In [2]:
# Load gene embedded coordinates.
plot_data_df = pd.read_csv(app_config.params['plot_data_df_path'][0], sep="\t", index_col=False)
# graph_adj = sp.sparse.load_npz(app_config.params['adj_mat_path'])
data_ess = pd.read_csv(app_config.params['raw_ess_data_path'], index_col=0, header=0, sep='\t')
data_ess = data_ess[data_ess.columns[:-4]]   # Only first 481 cols put through GLS, so isolate these

point_names = np.array(plot_data_df['gene_names'])
feat_names = data_ess.columns
cancer_types = data_ess.columns.str.split('_').str[1:].str.join(' ').str.capitalize().str.replace('Haematopoietic and lymphoid tissue', 'Hematopoietic/lymphoid')
additional_colorvars = []#app_config.params['additional_colorvars']

raw_data = data_ess.values

In [3]:
gene_names_df2 = pd.read_csv(app_config.params['gene_fullnames_path'], sep="\t", index_col=False)
a = dict(zip(gene_names_df2['symbol'], gene_names_df2['name']))
b = dict(zip(gene_names_df2['symbol'], gene_names_df2['ensembl_gene_id']))
full_point_names = np.copy(point_names)
ens_IDs = np.copy(point_names)

for i in range(len(point_names)):
    if point_names[i] in a:
        full_point_names[i] = a[point_names[i]]
        ens_IDs[i] = b[point_names[i]]

In [9]:
tmem_geneset = ['PTAR1', 
'COG7', 
'COG5', 
'C10ORF76', 
'P14KB', 
'TM9SF2', 
'COL4A3BP', 
'SGMS1', 
'KDSR', 
'SPTLC1', 
'SPTLC2', 
'SPTSSA', 
'CERS2', 
'ELOVL1', 
'TMEM189', 
'SEC14L1', 
'GNPAT', 
'AGPS', 
'FAR1', 
'PEX7', 
'EPT1', 
'PCYT2']

In [19]:
np.sum(np.isin(point_names, tmem_geneset))

20

In [32]:
pd.DataFrame(full_point_names).to_csv(app_config.params['genenames_path'], header=False, index=False)
pd.DataFrame(ens_IDs).to_csv(app_config.params['gene_ensID_path'], header=False, index=False)

In [4]:
itime = time.time()
print(time.time() - itime)
mutation_df = pd.read_csv(app_config.params['mutation_data_path'], sep="\t", index_col=False)
print(time.time() - itime)
shRNA_df = pd.read_csv(app_config.params['shRNA_data_path'], sep=",", index_col=0)
print(time.time() - itime)
expr_df = pd.read_csv(app_config.params['expression_data_path'], sep="\t", skiprows=2, index_col=False)
print(time.time() - itime)

0.0004029273986816406


/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning:

Columns (3,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.



6.214422941207886
8.801006078720093
23.94826579093933


In [104]:
geneview_data = mutation_df
mutdata = geneview_data[geneview_data[:, 0] == 'TP53', :]

TypeError: unhashable type: 'slice'

In [105]:
mutation_df.columns   #[['dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript', 'Tumor_Sample_Barcode', 'cDNA_Change', 'Codon_Change', 'Protein_Change', 'isDeleterious', 'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot', 'COSMIChsCnt', 'ExAC_AF', 'WES_AC', 'SangerWES_AC']]

Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1', 'dbSNP_RS',
       'dbSNP_Val_Status', 'Genome_Change', 'Annotation_Transcript',
       'Tumor_Sample_Barcode', 'cDNA_Change', 'Codon_Change', 'Protein_Change',
       'isDeleterious', 'isTCGAhotspot', 'TCGAhsCnt', 'isCOSMIChotspot',
       'COSMIChsCnt', 'ExAC_AF', 'WES_AC', 'SangerWES_AC',
       'SangerRecalibWES_AC', 'RNAseq_AC', 'HC_AC', 'RD_AC', 'WGS_AC',
       'Broad_ID'],
      dtype='object')

In [71]:
importlib.reload(app_config)
### noexpr_genes = point_names[np.where(~np.isin(point_names, expr_df['Description']))[0]]
# expr_arr = np.array(expr_df.iloc[:, 1:])
# arr_ndces = np.isin(expr_arr[:,0], point_names)
# np.save(app_config.params['expression_arr_path'], expr_arr[arr_ndces,:])
# np.save(app_config.params['expression_cell_lines_path'], [c.split(' (ACH')[0] for c in expr_df.columns[2:]])
# expr_arr = np.load(app_config.params['expression_arr_path'])
# expr_cell_lines = np.load(app_config.params['expression_cell_lines_path'])
# np.save(app_config.params['expression_arr_path'], np.vstack((np.array(['Gene'] + list(expr_cell_lines)), expr_arr)))

In [85]:
#expr_arr = np.load(app_config.params['expression_arr_path'])
exprdata_celllines = expr_arr[0, :]#[expr_arr[:, 0] == 'TP53', :]

In [87]:
np.array(expr_arr[:, exprdata_celllines == 'T24_URINARY_TRACT']).flatten()

array(['T24_URINARY_TRACT', 0.0, 0.0, ..., 6.9847, 0.0, 3.97005],
      dtype=object)

In [131]:
row_ndces = []
for g in point_names:
    if g in expr_df['Description']:
        row_ndces.append(np.where(expr_df['Description'] == g)[0][0])
col_ndces = []
for g in feat_names:
    if g in expr_df['Description']:
        col_ndces.append(np.where(expr_df['Description'] == g)[0][0])

(56318,)

In [97]:
nornai_feats = feat_names[np.where(~np.isin(feat_names, shRNA_df.columns))[0]]
nornai_genes = point_names[np.where(~np.isin(point_names, [c.split(' (')[0] for c in shRNA_df.index]))[0]]

In [106]:
# Save an abbreviated version of the mutation data, for speed reasons.
mutation_arr = np.array(mutation_df[['Hugo_Symbol', 'Chromosome', 'Start_position', 'End_position', 'Variant_Classification', 'Variant_Type', 
                                     'Tumor_Sample_Barcode', 'isDeleterious', 'isTCGAhotspot', 'isCOSMIChotspot', 'Codon_Change', 'Protein_Change']])
np.save(app_config.params['mutation_arr_path'], mutation_arr)

In [114]:
mutdata = mutation_arr[mutation_arr[:, 0] == 'TP53', :]
mutdata[:, [1,2,4,10,11]]
#where_mutations = np.isin(feat_names, xd)

array([['17', 7576854, 'Splice_Site', 'c.(991-993)cAg>cGg', 'p.Q331R'],
       ['17', 7577120, 'Missense_Mutation', 'c.(817-819)cGt>cAt',
        'p.R273H'],
       ['17', 7577536, 'Missense_Mutation', 'c.(745-747)Agg>Ggg',
        'p.R249G'],
       ...,
       ['17', 7577150, 'Splice_Site', 'c.(781-789)aggtagtgg>agg',
        'p.*W262fs'],
       ['17', 7578479, 'Frame_Shift_Del', 'c.(448-453)acacccfs',
        'p.P153fs'],
       ['17', 7578483, 'Frame_Shift_Del', 'c.(445-447)tccfs', 'p.S149fs']],
      dtype=object)

In [83]:
query_results = mutdata[(mutdata[:, 6] == feat_names[143]), :]
qwe = ["chr{}:{}    {}".format(
    query_results[i, 1], query_results[i, 2], query_results[i, 4]
    #, query_results[i, 7], query_results[i, 8], query_results[i, 9]
) for i in range(query_results.shape[0])]

In [23]:
gvdata = np.zeros_like(feat_names)
for i in range(len(feat_names)):
    if where_mutations[i]:
        gvdata[i] = 
# gvdata[np.isin(feat_names, xd)]

(331,)

# GO enrichment

In [46]:
from goatools.associations import read_ncbi_gene2go
from goatools.base import download_ncbi_associations
# Get ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz
gene2go = download_ncbi_associations()
go2geneids_human = read_ncbi_gene2go("gene2go", taxids=[9606], go2geneids=True)

from goatools.go_search import GoSearch
go2geneids_human = read_ncbi_gene2go("/Users/akshay/github/coessentiality-browser/gene2go", taxids=[9606], go2geneids=True)
srchhelp = GoSearch("data/go-basic.obo", go2items=go2geneids_human)

  wget.download(ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz out=gene2go.gz)


  gunzip gene2go.gz
  18,069 items READ: gene2go


In [102]:
gos = srchhelp.get_matching_gos(re.compile('0000003'))
mn = srchhelp.get_items(gos)

from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT


PATTERN SEARCH: "0000003"
MATCH GO:0030509(BMP signaling pathway) defn: "A series of molecular signals initiated by the binding of a member of the BMP (bone morphogenetic protein) family to a receptor on the surface of a target cell, and ending with regulation of a downstream cellular process, e.g. transcription." [GOC:signaling, ISBN:0878932437, PR:000000034]
MATCH GO:0036122(BMP binding) defn: "Interacting selectively and non-covalently with a member of the bone morphogenetic protein (BMP) family." [GOC:BHF, PMID:9660951, PR:000000034]
Found 2 GO(s) out of 18069 matching pattern("0000003")


In [121]:
from goatools import obo_parser
go_obo = 'data/go-basic.obo'
go = obo_parser.GODag(go_obo)
gotermIDs = []
gotermnames = []
for go_term in go.values():
    gotermIDs.append(str(go_term.id))
    gotermnames.append(str(go_term.name))

data/go-basic.obo: fmt(1.2) rel(2018-09-19) 47,343 GO Terms


In [122]:
np.save('gotermIDs.npy', np.array(gotermIDs))    
np.save('gotermnames.npy', np.array(gotermnames))


In [67]:
import goterm_caller
importlib.reload(gprofiler_caller)
gprofiler_caller.gprofiler(['GO:0000003'], mode='gconvert')
g = goterm_caller.gprofiler(['Klf4', 'Pax5', 'Sox2', 'Nanog', 'Jund', 'Max'])
g

AttributeError: 'NoneType' object has no attribute 'name'

In [57]:
ctypes = [x for x in np.unique(cancer_types)]
colorlist = ["#f7ff00","#fabebe","#ff8300","#f000ff","#74ee15","#4363d8","#001eff","#ff3300","#cab2d6","#008080","#808000","#bcf60c","#bec1d4","#fb9a99","#ffd8b1","#3cb44b","#bc5300","#ffe119","#33ccff","#911eb4","#46f0f0","#d220c8","#e6beff","#e6194b","#aaffc3","#000075"]
dict([x for x in zip(ctypes, colorlist[0:len(ctypes)])])

{'Autonomic ganglia': '#f7ff00',
 'Biliary tract': '#fabebe',
 'Bone': '#ff8300',
 'Breast': '#f000ff',
 'Central nervous system': '#74ee15',
 'Cervix': '#4363d8',
 'Endometrium': '#001eff',
 'Hematopoietic/lymphoid': '#ff3300',
 'Kidney': '#cab2d6',
 'Large intestine': '#008080',
 'Liver': '#808000',
 'Lung': '#bcf60c',
 'Oesophagus': '#bec1d4',
 'Ovary': '#fb9a99',
 'Pancreas': '#ffd8b1',
 'Pleura': '#3cb44b',
 'Skin': '#bc5300',
 'Soft tissue': '#ffe119',
 'Stomach': '#33ccff',
 'Thyroid': '#911eb4',
 'Upper aerodigestive tract': '#46f0f0',
 'Urinary tract': '#d220c8'}

# Layout of the graph
## coess_plotter to build the constituent graphs

In [6]:
itime = time.time()
for dparam in ['0.2', '0.5', '0.8', '0.9', '0.95', '0.99', 
               '0.8_batch_corrected', '0.9_batch_corrected', '0.95_batch_corrected', '0.99_batch_corrected']:
    gm_inc = sp.sparse.load_npz(_data_path + 'CO_d_{}_10NN.npz'.format(dparam))
    adj_mat = sp.sparse.csr_matrix((0.99 * gm_inc) + (0.01 * GLS_pvals_10))
    n_cmps = 100
    reduced_dim = 50
    vizdf_filename = _data_path + "vizdf_GLS01_CO99_d_{}.csv".format(dparam)

    # Create different eigenmaps: unweighted naive, and weighted diffmap
    emap_naive, eigvals = dm.diffmap_proj(adj_mat, t=0, n_comps=reduced_dim, embed_type='naive', return_eigvals=True)
    print("Laplacian eigenmap computed. Time: {}".format(time.time() - itime))
    emap_heat = dm.diffmap_proj(adj_mat, n_comps=n_cmps, n_dims=reduced_dim, min_energy_frac=0.9, embed_type='diffmap', return_eigvals=False)
    print("Diffusion components computed. Time: {}".format(time.time() - itime))
    
    ann_naive = adata.AnnData(X=emap_naive[:, :40])
    ann_heat = adata.AnnData(X=emap_heat[:, :40])
    
#     sc.tl.tsne(ann_heat, use_rep='X')#, perplexity=30)
#     print(time.time() - itime)
#     sc.tl.tsne(ann_naive, use_rep='X')
#     print(time.time() - itime)
    
    sc.pp.neighbors(ann_heat)
    print(time.time() - itime)
    
    itime = time.time()
    heat_umap = umap.UMAP().fit_transform(ann_heat.X)
    print(time.time() - itime)
    
#     naive_dm = emap_naive[:, :3].copy()
#     naive_tSNE = np.array(ann_naive.obsm['X_tsne'])
#     heat_dm = emap_heat[:, :3].copy()
#     heat_tSNE = np.array(ann_heat.obsm['X_tsne'])

    vizdf = pd.DataFrame(
        data=heat_umap, #np.hstack((# naive_dm, naive_tSNE, heat_dm, heat_tSNE, )), 
        columns=['hUMAP_x', 'hUMAP_y'], dtype='object')
    vizdf['gene_names'] = gene_names
    vizdf.to_csv(vizdf_filename, sep="\t", index=False)

NameError: name '_data_path' is not defined